In [1]:
import pandas as pd

df = pd.read_csv("../../complaints-2020-01-22_08_24.csv", encoding="utf-8")

df_selected = df.loc[:, ('Product', 'Consumer complaint narrative', 'Issue', 'State')]

---

__Functions to clean the Consumer complaint narrative__

In [2]:
import re
import string

def clean_url(complaint):
    # to do: more regex url garbage matching
    complaint = re.sub('https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', complaint)
    complaint = re.sub('https? ?: ?// ?(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', complaint)
    return complaint

# Remove punctuation from complaint
def clean_punctuation(complaint):
    complaint = re.sub('[%s]' % re.escape(string.punctuation), '', complaint)
    return complaint

# Remove non-sensical characters from complaint
def clean_nonsense(complaint):
    complaint = re.sub('[''""...]', '', complaint)
    complaint = re.sub('\n', '', complaint)
    return complaint

# Remove censored words from complaint
def clean_censored(complaint):
    complaint = re.sub('[XXXX]', '', complaint)
    return complaint

# Turn everything into lowercase
def clean_lowercase(complaint):
    complaint = complaint.lower()
    return complaint

# Remove numbers from complaint
def clean_numbers(complaint):
    complaint = re.sub('\w*\d\w', '', complaint)
    return complaint

__Application of narrative cleaning__

In [3]:
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_url)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_punctuation)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_nonsense)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_censored)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_lowercase)
df_selected["Consumer complaint narrative"] = df_selected["Consumer complaint narrative"].apply(clean_numbers)

**Apply CountVectorizer**

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

**Apply TF-IDF**

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

# Normalise with tf-idf
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print('Shape', X_train_tfidf.shape)
print(X_train_tfidf)

Shape (485701, 77541)
  (0, 77176)	0.057603117293145815
  (0, 76652)	0.16256155704199224
  (0, 75309)	0.0704998089772363
  (0, 75075)	0.06367724939199933
  (0, 74968)	0.09219266901371895
  (0, 73727)	0.12517299128438755
  (0, 71967)	0.17949182587880833
  (0, 69841)	0.11897824400517687
  (0, 69713)	0.05790316684384142
  (0, 69322)	0.11526692039409442
  (0, 69116)	0.0919644454692771
  (0, 67077)	0.0945540081983084
  (0, 66813)	0.13729499089431052
  (0, 65093)	0.10163692275565259
  (0, 63443)	0.12076334352602229
  (0, 62631)	0.062129528417944724
  (0, 62615)	0.12209465027976771
  (0, 62146)	0.11073198116064686
  (0, 61559)	0.07623732630893473
  (0, 59674)	0.09011779388563108
  (0, 59664)	0.09770742191644613
  (0, 58514)	0.05250540217562024
  (0, 58403)	0.04014663511243601
  (0, 57662)	0.14368581222288473
  (0, 56871)	0.14107643748845286
  :	:
  (485700, 72656)	0.20663930774149367
  (485700, 71225)	0.26283911340165067
  (485700, 70092)	0.0918765457662843
  (485700, 61605)	0.194740851743595

**Dimensionality Reduction**

In [6]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=5)

principal_components = tSVD.fit_transform(X_train_tfidf)
print(principal_components.shape)

(485701, 5)


__One hot encoding of extra columns__

In [7]:
principal_components_df = pd.DataFrame(principal_components)

df_selected['Issue'] = pd.Categorical(df_selected['Issue'])
df_dummies1 = pd.get_dummies(df_selected['Issue'])

df_selected['State'] = pd.Categorical(df_selected['State'])
df_dummies2 = pd.get_dummies(df_selected['State'], prefix = 'State')

df_concat = pd.concat([principal_components_df, df_dummies1, df_dummies2], axis = 1)
print(df_concat)

               0         1         2         3         4  \
0       0.187932  0.021534  0.008205 -0.001199 -0.031980   
1       0.123744  0.042946 -0.124335  0.076935 -0.010703   
2       0.177519  0.029225  0.167455 -0.002628 -0.023026   
3       0.196348  0.083256  0.259606  0.084695  0.074911   
4       0.277746  0.005665  0.134285  0.056163 -0.107760   
5       0.194631  0.027153  0.181898  0.019679 -0.040036   
6       0.155707  0.147171 -0.036024 -0.067435 -0.024959   
7       0.187515  0.221159 -0.050574 -0.070630  0.031529   
8       0.070227  0.066912 -0.026262 -0.018301 -0.024447   
9       0.194028 -0.193899  0.005108 -0.079831  0.034050   
10      0.186641  0.129005  0.132804  0.037158  0.063850   
11      0.187083 -0.099535  0.004441 -0.034304  0.009444   
12      0.206060 -0.112225 -0.076635 -0.002322  0.010096   
13      0.245870  0.154615  0.049834 -0.024651  0.026991   
14      0.200794  0.020068 -0.104357  0.045817  0.170108   
15      0.250801  0.159109 -0.170489  0.

**Split into train & test**

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(364275, 230)
(121426, 230)
(364275,)
(121426,)


**Run algorithm**

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

clf = RandomForestClassifier()
clf.fit(X_train, Y_train)

test_predictions = clf.predict(X_test)

**Report**

In [10]:
print(classification_report(test_predictions, Y_test))

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3621
                                                 Checking or savings account       0.99      1.00      1.00      4716
                                                               Consumer Loan       0.88      0.78      0.83      2648
                                                                 Credit card       0.99      0.97      0.98      4863
                                                 Credit card or prepaid card       0.95      0.98      0.97      7700
                                                            Credit reporting       1.00      1.00      1.00      7955
Credit reporting, credit repair services, or other personal consumer reports       1.00      0.97      0.98     36379
                                                       